# Hidden Markov Models

(You can find an extremely detailed and well organized review on HMMs [here](https://www.robots.ox.ac.uk/~vgg/rg/papers/hmm.pdf).)

A hidden Markov model is a [Markov chain](./Markov_Chains.ipynb) for which the state is only *partially observable*. In other words, observations are related to the state of the system, but they are typically insufficient to precisely determine the state.

The adjective *hidden* refers to the state sequence through which the model passes, not to the parameters of the model; the model is still referred to as a hidden Markov model even if these parameters are known exactly.

## Example: the genie with the urns
Consider this example: in a room that is not visible to an observer there is a genie. The room contains urns $X_1$, $X_2$, $X_3$, … each of which contains a known mix of balls, each ball labeled $y_1$, $y_2$, $y_3$, … . 

The genie chooses an urn in that room and randomly draws a ball from that urn. It then puts the ball onto a conveyor belt, where the observer can observe the sequence of the balls *but not the sequence of urns from which they were drawn*. The genie has some procedure to choose urns; the choice of the urn for the $n$-th ball depends *only upon a random number and the choice of the urn for the $(n − 1)$-th ball*. The choice of urn does not directly depend on the urns chosen before this single previous urn; therefore, this is called a Markov process.

## Architecture
We can visulaize an HMM as follows:

![](images/hmm_example.png)

where:

* $X_i$ are the states
* $y_k$ are the possible observations
* $a_{ij}$ are the state transition probabilities
* $b_{ik}$ are the output probabilities

We can also display the model dynamics with the following image:

![](images/hmm_seq.png)

In the standard type of hidden Markov model considered here, the state space of the hidden variables is discrete, while the observations themselves can either be discrete (typically generated from a categorical distribution) or continuous (typically from a Gaussian distribution). The parameters of a hidden Markov model are of two types, *transition probabilities* and *emission probabilities* (also known as output probabilities). The transition probabilities control the way the hidden state at time $t$ is chosen given the hidden state at time $t-1$.

If we have $N$ possible hidden states, then we will have a $N\times N$ transition matrix, which we have already explained in the [Markov chain](./Markov_Chains.ipynb) chapter. 

In addition, for each of the $N$ possible states, there is a set of emission probabilities governing the distribution of the observed variable at a particular time given the state of the hidden variable at that time. The size of this set depends on the nature of the observed variable.

## Probability of an observed sequence
A possible task, given an HMM, is to compute in a best way, given the parameters of the model, the probability of a particular output sequence. This requires summation over all possible state sequences:

The probability of a sequence $Y = y(0),y(1),...,y(L-1)$ is
$$
P(Y) = \sum_XP(Y | X)P(X),\quad \forall X = x(0),x(1),...,x(L-1)
$$

### Example
We are given the following HMM:

![](images/hmm_dry_rain_example.PNG)

Suppose we want to calculate a probability of a sequence of observations in our example $\{Dry,Rain\}$. We have to consider all the possibile hidden state sequences:

$$
\begin{align}
P(\{Dry,Rain\}) = & P(\{Dry,Rain\} , \{Low,Low\}) + \\
                & P(\{Dry,Rain\} , \{Low,High\}) + \\
                & P(\{Dry,Rain\} , \{High,Low\}) + \\ 
                & P(\{Dry,Rain\} , \{High,High\})
\end{align}
$$

where the first term is 

$$
\begin{align}
P(\{Dry,Rain\} , \{Low,Low\}) & = P(\{Dry,Rain\} | \{Low,Low\}) P(\{Low,Low\})\\
                              & = P(\{Dry | Low\})P(\{Rain | Low\})P(\{Low | Low\})P(\{Low\})\\
                              & = 0.4\times0.4\times0.6\times0.4\times0.3
\end{align}
$$

Of course this is a very impractical approach, since for a sequence of $k$ observations, one should scan all possible $N^k$ hidden state sequences. There is a way better approach.

### Forward-Backward HMM algorithm
We define the *forward variable* $\alpha_k(i)$ as the joint probability of the *partial* observation sequence $o_1, o_2, ..., o_k$ and that the hidden state at time $k$ is $S_i$. Therefore
$$
\alpha_k(i) = P(o_1, o_2, ..., o_k, q_k = S_i)
$$

![](images/hmm_forward_backward_algorithm.PNG)

#### Forward
The *forward* recursion for HMM is done as follows:
Given 
$$
\pi_i = P(S_i), \quad b_i(o_j) = P(o_j | S_i),\quad a_{ij} = P(S_i | S_j)
$$
1. **Initialization**
$$
\alpha_k(i) = P(o_1, q_1 = S_i) = \pi_i b_i(o_1), \quad i \in \{1, ..., N\},
$$
2. **Forward recursion**
$$
\begin{align}
\alpha_{k + 1}(j) & = P(o_1, o_2, ..., o_{k + 1}, q_{k + 1} = S_j) \\
                  & = \sum_i  P(o_1, o_2, ..., o_{k + 1}, q_k = S_i , q_{k + 1} = S_j) \\
                  & = \sum_i  P(o_1, o_2, ..., o_{k}, q_k = S_i) a_{ij}b_j(o_{k + 1}) \\
                  & = \left(\sum_i \alpha_k(i)a_{ij} \right)b_j(o_{k + 1}), \quad j \in \{1, ..., N\}, k \in \{1, ..., K-1\}
\end{align}
$$
3. **Termination**
$$
P(o_1, o_2, ..., o_K) = \sum_i P(o_1, o_2, ..., o_{K}, q_K = S_i) = \sum_i \alpha_K(i)
$$

In [10]:
import numpy as np

In [3]:
states = ('Healthy', 'Fever')
end_state = 'E'
 
observations = ('normal', 'cold', 'dizzy') 

start_probability = {'Healthy': 0.6, 'Fever': 0.4}

transition_probability = {
   'Healthy' : {'Healthy': 0.69, 'Fever': 0.3, 'E': 0.01},
   'Fever' : {'Healthy': 0.4, 'Fever': 0.59, 'E': 0.01},
}

emission_probability = {
   'Healthy' : {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
   'Fever' : {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6},
}

In [9]:
def forward(observations, states, start_prob, trans_prob, emm_prob, end_st):
    fwd = []
    f_prev = {}
    # we iterate on each observation o_i in order
    for i, observation_i in enumerate(observations):
        f_curr = {}
        # for every state st
        for st in states:
            if i == 0:
                # if we are at the beginning of the sequence, we can only  rely on the starting probabilities
                # of the states
                prev_f_sum = start_prob[st]
            else:
                # if we are in the middle of the sequence, the compute the k-th value of a_k
                # by summing up the probabilities of transition form the previous states to state
                # st multiplied by the previous a_k 
                prev_f_sum = sum(f_prev[k]*trans_prob[k][st] for k in states)
            # we set the value for state st as the emission probability of observation i in state st
            # times the factor computed at the previous step
            f_curr[st] = emm_prob[st][observation_i] * prev_f_sum #pi_s * B_s(o_i)
        fwd.append(f_curr)
        # the current a_k state values become the previous for the next iteration
        f_prev = f_curr
    # finally, once we have seen all the observation sequence, the final probability is 
    # the summation of the a_k times the transition probability to the end state
    p_fwd = sum(f_curr[k] * trans_prob[k][end_st] for k in states)
    return p_fwd, fwd

In [8]:
forward(observations, states, start_probability, transition_probability, emission_probability, end_state)

(0.00035638319999999995,
 [{'Fever': 0.04000000000000001, 'Healthy': 0.3},
  {'Fever': 0.03408, 'Healthy': 0.0892},
  {'Fever': 0.028120319999999997, 'Healthy': 0.007518}])

It must be noticed that the forward algorithm enables also to compute the *joint probability* of $P(o_1, o_2, ..., o_K, q_K = S_i)$ thus enabling us to discover, at the end of the analysis of the input sequence, the probability of the various states.

#### Backward
Similarily to the forward pass, we now define a backward variable $\beta_k(i)$ which is the joint probability of the *partial* observation sequence $o_{k + 1}, o_{k + 2}, ..., o_{K}$ given that the hidden state at time $k$ is $S_i$:

$$
\beta_k(i) = P(o_{k + 1}, o_{k + 2}, ..., o_{K} | q_k = S_i)
$$

1. **Initialization**
$$
\beta_k(i) = 1, \quad i \in \{1, ..., N\}
$$

2. **Backward recursion**
$$
\begin{align}
\beta_{k}(j) & = P(o_{k + 1}, o_{k + 2}, ..., o_{K} | q_{k} = S_j) \\
                  & = \sum_i  P(o_{k + 1}, o_{k + 2}, ..., o_{K}, q_{k + 1} = S_i | q_{k} = S_j) \\
                  & = \sum_i  P(o_{k + 2}, o_{k + 3}, ..., o_{K} | q_{k + 1} = S_i) a_{ji}b_i(o_{k + 1}) \\
                  & = \sum_i \beta_{k + 1}(i)a_{ji}b_i(o_{k+1}), \quad j \in \{1, ..., N\}, k \in \{1, ..., K - 1\}
\end{align}
$$

3. **Termination**
$$
\begin{align}
P(o_1, o_2, ..., o_K) & = \sum_i P(o_1, o_2, ..., o_K, q_1 = S_i)\\
                      & = \sum_i P(o_1, o_2, ..., o_K | q_1 = S_i)P(q_1 = S_i)\\
                      & = \sum_i \beta_1(i)b_i(o_1)\pi_i
\end{align}
$$

## Probability of the latent variables
An other possible task is, given an HMM and a sequence of observations $y(1),...,y(t)$, to infer some information about the latent variables. This can be divided in several subtasks as follows.

### Filtering
The task is to compute, given the model's parameters and a sequence of observations, the distribution over hidden states of the last latent variable at the end of the sequence
$$
P(x(t) | y(1),...,y(t))
$$

### Smoothing
This is similar to filtering but asks about the distribution of a latent variable somewhere in the middle of a sequence
$$
P(x(k) | y(1),...,y(t)), \quad k < t
$$

### Most likely explanation
The task, unlike the previous two, asks about the joint probability of the entire sequence of hidden states that generated a particular sequence of observations. This task is generally applicable when HMM's are applied to different sorts of problems from those for which the tasks of filtering and smoothing are applicable. An example is *part-of-speech tagging*, where the hidden states represent the underlying parts of speech corresponding to an observed sequence of words. In this case, what is of interest is the entire sequence of parts of speech, rather than simply the part of speech for a single word, as filtering or smoothing would compute.

## Learning the HMM parameters
The parameter learning task in HMMs is to find, given an output sequence or a set of such sequences, the best set of state transition and emission probabilities. The task is usually to derive the maximum likelihood estimate of the parameters of the HMM given the set of output sequences. No tractable algorithm is known for solving this problem exactly, but a local maximum likelihood can be derived efficiently using the Baum–Welch algorithm or the Baldi–Chauvin algorithm.

### Baum–Welch algorithm

In [1]:
4+4

8